In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 64 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 21 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [3]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [4]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VHc.csv', delimiter=',')

In [5]:
data = my_data[1:,24:]
print(data.shape)

(291, 21)


In [6]:
print(data)

[[0.01813659 0.02478316 0.00467864 ... 0.04624293 0.05025645 0.00981916]
 [0.04035907 0.0454523  0.0054114  ... 0.02561918 0.02236751 0.00540298]
 [0.01133233 0.01657404 0.01011706 ... 0.08537867 0.03078398 0.00810179]
 ...
 [0.00976569 0.01256512 0.02513162 ... 0.04750742 0.04829812 0.01046296]
 [0.03669534 0.06059309 0.02861252 ... 0.03881095 0.04491708 0.03071279]
 [0.03060335 0.05044421 0.01327568 ... 0.04509031 0.04975136 0.04267977]]


In [7]:
print(data.max())

0.142512947320938


In [8]:
data = np.array(data)
print(data.shape)

(291, 21)


In [9]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [10]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [11]:
p = np.array(ctype)
print(p.shape)

(292,)


In [12]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [13]:
label = np.array(label)
print(label.shape)

(291,)


In [14]:
# 80% train 20% test (NDVIclean)
X_train = data[0:232,:]
X_test = data[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 21)
(59, 21)
(232,)
(59,)


In [21]:
print(X_train[0])

[0.01813659 0.02478316 0.00467864 0.00802058 0.00614472 0.00813453
 0.01108583 0.02748296 0.02118975 0.06373059 0.03225798 0.02828747
 0.03696234 0.02951577 0.03866692 0.01870059 0.02247578 0.04234596
 0.04624293 0.05025645 0.00981916]


In [15]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [16]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.87175784
Epoch: 2 cost = 0.86168709
Epoch: 3 cost = 0.85144654
Epoch: 4 cost = 0.84070356
Epoch: 5 cost = 0.82921844
Epoch: 6 cost = 0.81681886
Epoch: 7 cost = 0.80338120
Epoch: 8 cost = 0.78881726
Epoch: 9 cost = 0.77306852
Epoch: 10 cost = 0.75610709
Epoch: 11 cost = 0.73794345
Epoch: 12 cost = 0.71863723
Epoch: 13 cost = 0.69830755
Epoch: 14 cost = 0.67713119
Epoch: 15 cost = 0.65532125
Epoch: 16 cost = 0.63308287
Epoch: 17 cost = 0.61056265
Epoch: 18 cost = 0.58781543
Epoch: 19 cost = 0.56480166
Epoch: 20 cost = 0.54139579
Epoch: 21 cost = 0.51735300
Epoch: 22 cost = 0.49222490
Epoch: 23 cost = 0.46555704
Epoch: 24 cost = 0.43879669
Epoch: 25 cost = 0.41744273
Epoch: 26 cost = 0.40425510
Epoch: 27 cost = 0.39684032
Epoch: 28 cost = 0.39287305
Epoch: 29 cost = 0.39077051
Epoch: 30 cost = 0.38954874
Epoch: 31 cost = 0.38870208
Epoch: 32 cost = 0.38800346
Epoch: 33 cost = 0.38735269
Epoch: 34 cost = 0.38669494
Epoch: 35 cost = 0.38598317
Epoch: 36 cost = 0.38515755
E

Epoch: 410 cost = 0.04943423
Epoch: 411 cost = 0.04804046
Epoch: 412 cost = 0.04849413
Epoch: 413 cost = 0.04880216
Epoch: 414 cost = 0.04819878
Epoch: 415 cost = 0.04759656
Epoch: 416 cost = 0.04883239
Epoch: 417 cost = 0.05432350
Epoch: 418 cost = 0.05699248
Epoch: 419 cost = 0.04647881
Epoch: 420 cost = 0.04615743
Epoch: 421 cost = 0.04726351
Epoch: 422 cost = 0.04650985
Epoch: 423 cost = 0.04508891
Epoch: 424 cost = 0.04553309
Epoch: 425 cost = 0.04769405
Epoch: 426 cost = 0.05097607
Epoch: 427 cost = 0.04990443
Epoch: 428 cost = 0.04430661
Epoch: 429 cost = 0.04452169
Epoch: 430 cost = 0.04625419
Epoch: 431 cost = 0.04489594
Epoch: 432 cost = 0.04266352
Epoch: 433 cost = 0.04379806
Epoch: 434 cost = 0.04608835
Epoch: 435 cost = 0.04806524
Epoch: 436 cost = 0.04609086
Epoch: 437 cost = 0.04194458
Epoch: 438 cost = 0.04262299
Epoch: 439 cost = 0.04403007
Epoch: 440 cost = 0.04251429
Epoch: 441 cost = 0.04063819
Epoch: 442 cost = 0.04203234
Epoch: 443 cost = 0.04395314
Epoch: 444 cos

Epoch: 698 cost = 0.00301535
Epoch: 699 cost = 0.00311815
Epoch: 700 cost = 0.00292749
Epoch: 701 cost = 0.00271052
Epoch: 702 cost = 0.00266898
Epoch: 703 cost = 0.00245957
Epoch: 704 cost = 0.00239329
Epoch: 705 cost = 0.00233799
Epoch: 706 cost = 0.00229079
Epoch: 707 cost = 0.00250627
Epoch: 708 cost = 0.00291377
Epoch: 709 cost = 0.01011136
Epoch: 710 cost = 0.02469498
Epoch: 711 cost = 0.00257906
Epoch: 712 cost = 0.00230841
Epoch: 713 cost = 0.00266406
Epoch: 714 cost = 0.00236343
Epoch: 715 cost = 0.00273064
Epoch: 716 cost = 0.00241215
Epoch: 717 cost = 0.00280514
Epoch: 718 cost = 0.00248477
Epoch: 719 cost = 0.00293710
Epoch: 720 cost = 0.00256283
Epoch: 721 cost = 0.00297702
Epoch: 722 cost = 0.00257388
Epoch: 723 cost = 0.00272972
Epoch: 724 cost = 0.00246929
Epoch: 725 cost = 0.00235671
Epoch: 726 cost = 0.00222656
Epoch: 727 cost = 0.00199367
Epoch: 728 cost = 0.00193560
Epoch: 729 cost = 0.00175333
Epoch: 730 cost = 0.00174613
Epoch: 731 cost = 0.00166926
Epoch: 732 cos

Epoch: 983 cost = 0.00020331
Epoch: 984 cost = 0.00019538
Epoch: 985 cost = 0.00018987
Epoch: 986 cost = 0.00018326
Epoch: 987 cost = 0.00017795
Epoch: 988 cost = 0.00017313
Epoch: 989 cost = 0.00017010
Epoch: 990 cost = 0.00016767
Epoch: 991 cost = 0.00016506
Epoch: 992 cost = 0.00016309
Epoch: 993 cost = 0.00016205
Epoch: 994 cost = 0.00016052
Epoch: 995 cost = 0.00016039
Epoch: 996 cost = 0.00015767
Epoch: 997 cost = 0.00015534
Epoch: 998 cost = 0.00015621
Epoch: 999 cost = 0.00015457
Epoch: 1000 cost = 0.00015348


In [17]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
print(y_test.shape)

(59, 5)


In [19]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7288136


In [20]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data[0:204,:]
X_test = data[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 21)
(87, 21)
(204,)
(87,)


In [116]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [120]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.31313014
Epoch: 2 cost = 1.28349408
Epoch: 3 cost = 1.25635783
Epoch: 4 cost = 1.23073848
Epoch: 5 cost = 1.20587925
Epoch: 6 cost = 1.18121755
Epoch: 7 cost = 1.15634950
Epoch: 8 cost = 1.13098903
Epoch: 9 cost = 1.10492563
Epoch: 10 cost = 1.07799117
Epoch: 11 cost = 1.05003691
Epoch: 12 cost = 1.02092493
Epoch: 13 cost = 0.99052509
Epoch: 14 cost = 0.95871274
Epoch: 15 cost = 0.92535865
Epoch: 16 cost = 0.89031800
Epoch: 17 cost = 0.85343583
Epoch: 18 cost = 0.81463333
Epoch: 19 cost = 0.77421161
Epoch: 20 cost = 0.73341930
Epoch: 21 cost = 0.69459848
Epoch: 22 cost = 0.65991588
Epoch: 23 cost = 0.62998505
Epoch: 24 cost = 0.60416790
Epoch: 25 cost = 0.58148058
Epoch: 26 cost = 0.56103994
Epoch: 27 cost = 0.54218690
Epoch: 28 cost = 0.52451392
Epoch: 29 cost = 0.50786386
Epoch: 30 cost = 0.49229612
Epoch: 31 cost = 0.47800112
Epoch: 32 cost = 0.46518185
Epoch: 33 cost = 0.45395803
Epoch: 34 cost = 0.44434591
Epoch: 35 cost = 0.43631601
Epoch: 36 cost = 0.42982411
E

Epoch: 341 cost = 0.10691121
Epoch: 342 cost = 0.10655226
Epoch: 343 cost = 0.10641096
Epoch: 344 cost = 0.10596722
Epoch: 345 cost = 0.10519993
Epoch: 346 cost = 0.10445727
Epoch: 347 cost = 0.10383968
Epoch: 348 cost = 0.10316582
Epoch: 349 cost = 0.10274676
Epoch: 350 cost = 0.10250151
Epoch: 351 cost = 0.10203519
Epoch: 352 cost = 0.10131330
Epoch: 353 cost = 0.10061157
Epoch: 354 cost = 0.10002836
Epoch: 355 cost = 0.09924828
Epoch: 356 cost = 0.09882073
Epoch: 357 cost = 0.09861127
Epoch: 358 cost = 0.09814458
Epoch: 359 cost = 0.09734445
Epoch: 360 cost = 0.09657622
Epoch: 361 cost = 0.09588031
Epoch: 362 cost = 0.09510927
Epoch: 363 cost = 0.09466166
Epoch: 364 cost = 0.09440444
Epoch: 365 cost = 0.09393050
Epoch: 366 cost = 0.09308328
Epoch: 367 cost = 0.09216807
Epoch: 368 cost = 0.09131741
Epoch: 369 cost = 0.09069906
Epoch: 370 cost = 0.09045211
Epoch: 371 cost = 0.09032122
Epoch: 372 cost = 0.08964970
Epoch: 373 cost = 0.08857429
Epoch: 374 cost = 0.08771276
Epoch: 375 cos

Epoch: 655 cost = 0.00993136
Epoch: 656 cost = 0.01040780
Epoch: 657 cost = 0.00971196
Epoch: 658 cost = 0.01020074
Epoch: 659 cost = 0.00948942
Epoch: 660 cost = 0.00998151
Epoch: 661 cost = 0.00926114
Epoch: 662 cost = 0.00976779
Epoch: 663 cost = 0.00905467
Epoch: 664 cost = 0.00956990
Epoch: 665 cost = 0.00886321
Epoch: 666 cost = 0.00937971
Epoch: 667 cost = 0.00867193
Epoch: 668 cost = 0.00918668
Epoch: 669 cost = 0.00847482
Epoch: 670 cost = 0.00899211
Epoch: 671 cost = 0.00827561
Epoch: 672 cost = 0.00879232
Epoch: 673 cost = 0.00807874
Epoch: 674 cost = 0.00860158
Epoch: 675 cost = 0.00789520
Epoch: 676 cost = 0.00841599
Epoch: 677 cost = 0.00771553
Epoch: 678 cost = 0.00824406
Epoch: 679 cost = 0.00754044
Epoch: 680 cost = 0.00806165
Epoch: 681 cost = 0.00736734
Epoch: 682 cost = 0.00788333
Epoch: 683 cost = 0.00719171
Epoch: 684 cost = 0.00770524
Epoch: 685 cost = 0.00702120
Epoch: 686 cost = 0.00752654
Epoch: 687 cost = 0.00684998
Epoch: 688 cost = 0.00736116
Epoch: 689 cos

Epoch: 964 cost = 0.00016792
Epoch: 965 cost = 0.00016694
Epoch: 966 cost = 0.00016316
Epoch: 967 cost = 0.00016067
Epoch: 968 cost = 0.00016174
Epoch: 969 cost = 0.00015750
Epoch: 970 cost = 0.00015831
Epoch: 971 cost = 0.00015681
Epoch: 972 cost = 0.00015340
Epoch: 973 cost = 0.00015031
Epoch: 974 cost = 0.00014875
Epoch: 975 cost = 0.00014833
Epoch: 976 cost = 0.00014796
Epoch: 977 cost = 0.00014632
Epoch: 978 cost = 0.00014375
Epoch: 979 cost = 0.00014361
Epoch: 980 cost = 0.00014555
Epoch: 981 cost = 0.00014409
Epoch: 982 cost = 0.00014077
Epoch: 983 cost = 0.00014036
Epoch: 984 cost = 0.00014143
Epoch: 985 cost = 0.00014212
Epoch: 986 cost = 0.00014060
Epoch: 987 cost = 0.00013954
Epoch: 988 cost = 0.00013870
Epoch: 989 cost = 0.00013963
Epoch: 990 cost = 0.00014160
Epoch: 991 cost = 0.00014782
Epoch: 992 cost = 0.00018443
Epoch: 993 cost = 0.00324778
Epoch: 994 cost = 0.00573820
Epoch: 995 cost = 0.00058567
Epoch: 996 cost = 0.00019045
Epoch: 997 cost = 0.00016991
Epoch: 998 cos

In [122]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7011494
